In [ ]:
import pandas as pd
import folium
import numpy as np
from plots.Folium_heat import *
from plots.Folium_heat_time import *
import webbrowser

In [ ]:
df = pd.read_pickle("data/final_final_data.pkl")
df.head()

In [ ]:
df.EnergyLabelClassification.unique()

In [ ]:
labels=sorted(['D', 'A', 'E', 'C', 'F', 'G', 'B'])

In [ ]:
list(df.columns)

# Heatmap plots for energy labels

In [ ]:
#all
create_folium_heatmap(df,"Wgs84Latitude","Wgs84Longitude",3,10)

In [ ]:
create_folium_heatmap(df[df.EnergyLabelClassification == "A"],"Wgs84Latitude","Wgs84Longitude",3,5)

In [ ]:
create_folium_heatmap(df[df.EnergyLabelClassification == "B"],"Wgs84Latitude","Wgs84Longitude",3,5)

In [ ]:
create_folium_heatmap(df[df.EnergyLabelClassification == "C"],"Wgs84Latitude","Wgs84Longitude",3,5)

In [ ]:
create_folium_heatmap(df[df.EnergyLabelClassification == "D"],"Wgs84Latitude","Wgs84Longitude",3,5)

In [ ]:
create_folium_heatmap(df[df.EnergyLabelClassification == "E"],"Wgs84Latitude","Wgs84Longitude",3,5)

In [ ]:
create_folium_heatmap(df[df.EnergyLabelClassification == "F"],"Wgs84Latitude","Wgs84Longitude",3,5)

In [ ]:
create_folium_heatmap(df[df.EnergyLabelClassification == "G"],"Wgs84Latitude","Wgs84Longitude",3,5)

# Dist of energylabels for new buildings

In [ ]:
list(df.columns)

In [ ]:
df.iloc[11006].dateOfRenovation_dateOfEvent_anyPoint

In [ ]:
df[df.YearOfConstruction =="1000"][["Wgs84Latitude","Wgs84Longitude"]]

# Heatmaps roof types

In [ ]:
df.materialOfRoof.unique()

In [ ]:
create_folium_heatmap(df[df.materialOfRoof == "clay Tile"],"Wgs84Latitude","Wgs84Longitude",3,5)

In [ ]:
create_folium_heatmap(df[df.materialOfRoof == "concrete Tile"],"Wgs84Latitude","Wgs84Longitude",3,5)

In [ ]:
create_folium_heatmap(df[df.materialOfRoof == "vegetated Green Roof"],"Wgs84Latitude","Wgs84Longitude",3,5)

In [ ]:
years=df.dateOfRenovation_dateOfEvent_anyPoint.unique()
map_data=[[[row['Wgs84Latitude'],row['Wgs84Longitude']] for index, row in df[df.dateOfRenovation_dateOfEvent_anyPoint==i].iterrows()] for i in years]

# Heatmap Time

In [ ]:
create_folium_heatmap_time("YearOfConstruction",'Wgs84Latitude','Wgs84Longitude',df)

In [ ]:
df.dateOfRenovation_dateOfEvent_anyPoint[0]

In [ ]:
df.Kvadratmeterpris.unique()

In [ ]:
import pandas_alive

In [ ]:
db=pd.crosstab(index=df['YearOfConstruction'], columns=df['EnergyLabelClassification']).cumsum()

In [ ]:
db2=pd.crosstab(index=df.ValidFrom.dt.to_period('M'), columns=df['EnergyLabelClassification']).cumsum()

In [ ]:
db2=db2.reset_index()

In [ ]:
db.index

In [ ]:
db2.fillna(0).plot_animated(filename='example-line-chart2.gif',kind='line',period_label={'x':0.25,'y':0.9})

In [ ]:
db.plot_animated(filename='example-line-chart.gif',kind='line',period_label={'x':0.25,'y':0.9})

# Predictions

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import *

In [ ]:
def metrics(Y_test,y_pred,x_val,model):
    y_pred=y_pred
    x_val=x_val
    model=model
    auc=round(accuracy_score(Y_test,y_pred),4)
    f_score=round(f1_score(Y_test, y_pred),4)
    P_score=round(precision_score(Y_test,y_pred),4)
    recall=round(recall_score(Y_test, y_pred),4)
    
    ns_probs = [0 for _ in range(len(Y_test))]
    lr_probs = model.predict_proba(x_val)
    lr_probs = lr_probs[:, 1]
    # calculate scores
    ns_auc = roc_auc_score(Y_test, ns_probs)
    lr_auc = round(roc_auc_score(Y_test, lr_probs),4)
    return [auc,f_score,P_score,recall,lr_auc]

def plot_feature_importance(importance,names,model_type):

    #Create arrays from feature importance and feature names
    feature_importance = importance
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_df = fi_df[fi_df.feature_importance > 0.01]
    

    #Define size of bar plot
    plt.figure(figsize=(8,6))
    plt.tight_layout()
    plt.yticks(fontsize=7)
    #Plot Searborn bar chart
    ax=sns.barplot(y='feature_names',x='feature_importance',data=fi_df,palette="rocket")
    ax.set_xlabel('feature_importance')
    #plt.xticks(rotation = 90)
    
    #Add chart labels
    plt.title(model_type + ' FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE > 0.01')
    plt.ylabel('FEATURE NAMES')


def evaluation_time(y_test,y_pred,model,x_val):
    plt.style.use('ggplot')
    cm = confusion_matrix(y_test, (y_pred))
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax,fmt='g',cmap='rocket'); #annot=True to annotate cells

    # labels, title and ticks
    ax.set_xlabel('Predicted Winner');ax.set_ylabel('Actual Winner'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(['Server', 'Returner']); ax.yaxis.set_ticklabels(['Server', 'Returner'], rotation=360);
    plt.show()

    f_score=f1_score(y_test, y_pred, average=None)
    P_score=precision_score(y_test,y_pred,average=None)
    recall=recall_score(y_test, y_pred, average=None)


    ns_probs = [0 for _ in range(len(y_test))]
    lr_probs = model.predict_proba(x_val)
    lr_probs = lr_probs[:, 1]
    # calculate scores
    ns_auc = roc_auc_score(y_test, ns_probs)
    lr_auc = roc_auc_score(y_test, lr_probs)
    # summarize scores
    print(f"Test set accuracy score {(np.round(accuracy_score(y_pred,y_test),3)*100)}%")
    print(f"F1 score {np.round(f_score,3)}")
    print(f"Recall score {np.round(recall,3)}")
    print(f"Precision score {np.round(P_score,3)}")
    print('Model: ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
    lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='Base')
    plt.plot(lr_fpr, lr_tpr, marker='.', label='XGBoost')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.show()

def evaluation(y_test,y_pred,model_xg,x_val):
    plt.style.use('ggplot')
    cm = confusion_matrix(y_test, (y_pred))
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax,fmt='g',cmap='rocket'); #annot=True to annotate cells

    ax.set_xlabel('Predcited Winner');ax.set_ylabel('Actual Winner'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(['Server', 'Returner']); ax.yaxis.set_ticklabels(['Server', 'Returner'], rotation=360);
    plt.show()

    f_score=f1_score(y_test, y_pred, average=None)
    P_score=precision_score(y_test,y,average=None)
    recall=recall_score(y_test, y_pred, average=None)


    ns_probs = [0 for _ in range(len(y_test))]
    lr_probs = model_xg.predict_proba(x_val)
    lr_probs = lr_probs[:, 1]
    # calculate scores
    ns_auc = roc_auc_score(y_test, ns_probs)
    lr_auc = roc_auc_score(y_test, lr_probs)
    # summarize scores
    print(f"Test set accuracy score {np.round(val_scoreXG*100,3)}%")
    print(f"F1 score {np.round(f_score,3)}")
    print(f"Recall score {np.round(recall,3)}")
    print(f"Precision score {np.round(P_score,3)}")
    print('Xgboost: ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
    lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='Base')
    plt.plot(lr_fpr, lr_tpr, marker='.', label='XGBoost')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.show()
    
    results = model_xg.evals_result()
    epochs = len(results['validation_0']['error'])
    x_axis = range(0, epochs)
    # plot log loss
    fig, ax = plt.subplots()
    ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
    ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
    ax.legend()
    plt.ylabel('Log Loss')
    plt.title('XGBoost Log Loss')
    plt.show()
    # plot classification error
    fig, ax = plt.subplots()
    ax.plot(x_axis, results['validation_0']['error'], label='Train')
    ax.plot(x_axis, results['validation_1']['error'], label='Test')
    ax.legend()
    plt.ylabel('Classification Error')
    plt.title('XGBoost Classification Error')
    plt.show()
    return


In [ ]:
pred_df = df.drop_duplicates(subset=['addressRepresentation'])
pred_df.head()

In [ ]:
X,y=[
              'connectionToSewage',
              'connectionToWater',
              'currentUse',
              'dateOfConstruction_dateOfEvent_anyPoint',
              'dateOfRenovation_dateOfEvent_anyPoint',
              'HeatSupply',
              'materialOfRoof',
              'materialOfWall',
              'YearOfConstruction',
              ],['EnergyLabelClassification']

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
y=df[y].values
x=df[X]
x=pd.get_dummies(x)
labels=list(x.columns)
x=x.values
scaler = StandardScaler()
x=scaler.fit_transform(x)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

model_xg = XGBClassifier(objective ='binary:logistic')
model_xg.fit(X_train, y_train);
y_pred = model_xg.predict(X_test);

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm,
                     index = list(df.EnergyLabelClassification.unique()), 
                     columns = list(df.EnergyLabelClassification.unique()))

In [ ]:
#Plotting the confusion matrix
plt.figure(figsize=(20,10))
sns.heatmap(cm_df, annot=True)
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()

In [ ]:
scoreXG = accuracy_score(y_pred, y_test)
#evaluation(y_test,y_pred,model_xg,X_test)
plot_feature_importance(model_xg.feature_importances_,labels,"XG Second Serve ")